Import Packages

In [1]:
!pip3 install lxml
!pip3 install html5lib


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import html5lib

Getting HTML for web scraping

In [2]:
#reusable URLs to help us get the different categories that we want. 

pro_football_reference_main = 'https://www.pro-football-reference.com/years/'
most_recent_season = '2023'

passing_stats = '/passing.htm'
rushing_stats = '/rushing.htm'
receiving_stats = '/receiving.htm'
fantasy_stats = '/fantasy.htm'


def get_stats(url):
    """Downloads the web page and we will return a beautiful soup doc."""
    response = requests.get(url)
        
    # verify that we are receiving the correct response
    
    if response.status_code != 200:
        raise Exception(f'Unable to download page{url}')
        
    # Get the HTML from the page
    page_content = response.text
    
    # create our bs4 doc
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc

In [3]:
qb_stats = get_stats(pro_football_reference_main + most_recent_season + passing_stats)
qb_stats.find('title')


<title>2023 NFL Passing | Pro-Football-Reference.com</title>

Getting stats for QBs

In [11]:
def get_all_qbs(qb_stats):
    qbs = qb_stats.find('tbody').find_all('tr')
    # print(qbs)
    all_qbs_data = []
    for qb in qbs:
        
        try:
            all_stats = {
                'name': qb.find('td').find('a').text,
                'team': qb.find(attrs = {'data-stat' : 'pos'}).text,
                'pos' :     qb.find(attrs = {'data-stat' : 'team_name_abbr'}).text,
                'comp_pct': qb.find(attrs = {'data-stat' : 'pass_cmp_pct'}).text,
                'pass_yds': qb.find(attrs = {'data-stat' : 'pass_yds'}).text,
                'pass_td' : qb.find(attrs = {'data-stat' : 'pass_td'}).text,
                'pass_int' : qb.find(attrs = {'data-stat' : 'pass_int'}).text,
                'pass_td_%' : qb.find(attrs = {'data-stat': 'pass_td_pct'}).text,
                'QB_Rating' : qb.find(attrs = {'data-stat' : 'qbr'}).text,
                'Fourth_Qtr_Comebacks': qb.find(attrs = {'data-stat' : 'comebacks'}).text  
        }
        
        except:
            all_qbs_data.append({})
            print('No data here')
        all_qbs_data.append(all_stats)
    return all_qbs_data

In [22]:
get_all_qbs(qb_stats)

No data here


[{'name': 'Tua Tagovailoa',
  'team': 'QB',
  'pos': 'MIA',
  'comp_pct': '69.3',
  'pass_yds': '4624',
  'pass_td': '29',
  'pass_int': '14',
  'pass_td_%': '5.2',
  'QB_Rating': '60.8',
  'Fourth_Qtr_Comebacks': '2'},
 {'name': 'Jared Goff',
  'team': 'QB',
  'pos': 'DET',
  'comp_pct': '67.3',
  'pass_yds': '4575',
  'pass_td': '30',
  'pass_int': '12',
  'pass_td_%': '5.0',
  'QB_Rating': '60.3',
  'Fourth_Qtr_Comebacks': '2'},
 {'name': 'Dak Prescott',
  'team': 'QB',
  'pos': 'DAL',
  'comp_pct': '69.5',
  'pass_yds': '4516',
  'pass_td': '36',
  'pass_int': '9',
  'pass_td_%': '6.1',
  'QB_Rating': '72.7',
  'Fourth_Qtr_Comebacks': '2'},
 {'name': 'Josh Allen',
  'team': 'QB',
  'pos': 'BUF',
  'comp_pct': '66.5',
  'pass_yds': '4306',
  'pass_td': '29',
  'pass_int': '18',
  'pass_td_%': '5.0',
  'QB_Rating': '69.6',
  'Fourth_Qtr_Comebacks': '2'},
 {'name': 'Brock Purdy',
  'team': 'QB',
  'pos': 'SFO',
  'comp_pct': '69.4',
  'pass_yds': '4280',
  'pass_td': '31',
  'pass_int

In [25]:
def write_csv(items, path):
    #open the file in write mode
    print(items)
    with open(path, 'w') as f:
        #return nothing if there's nothing to write
        if len(items) == 0:
            return
        
        #write the headers in the first line of csv
        headers = list(items[0].keys())
        f.write(','.join(headers) +'\n')
        
        #write one item per line from our dictionary
        for item in items:
            values = []
            for header in headers:
                values.append(str(item.get(header, '')))
            f.write(','.join(values) + '\n')

In [14]:
write_csv(get_all_qbs(qb_stats), '../data/2023_qb_stats.csv')

No data here


In [15]:
team_stats = get_stats(pro_football_reference_main + most_recent_season)
team_stats.find('title')


<title>2023 NFL Standings &amp; Team Stats | Pro-Football-Reference.com</title>

In [59]:
def get_all_teams(team_stats):
    afc_teams = team_stats.find("div", {"id": "all_AFC"}).find('tbody').find_all('tr')
    print(afc_teams)

    nfc_teams = team_stats.find("div", {"id": "all_NFC"}).find('tbody').find_all('tr')
    print(nfc_teams)

    all_teams_data = []
    for team in afc_teams:
        try:
            all_stats = {
                'team': team.find('th', attrs={"data-stat": "team"}).find('a').text,
                'wins' :     team.find(attrs = {'data-stat' : 'wins'}).text,
                'win_loss_perc': team.find(attrs = {'data-stat' : 'win_loss_perc'}).text,
                'points': team.find(attrs = {'data-stat' : 'points'}).text,
                'points_opp' : team.find(attrs = {'data-stat' : 'points_opp'}).text,
                'points_diff' : team.find(attrs = {'data-stat' : 'points_diff'}).text,
                'mov' : team.find(attrs = {'data-stat': 'mov'}).text,
                'sos_total' : team.find(attrs = {'data-stat' : 'sos_total'}).text,
                'srs_total': team.find(attrs = {'data-stat' : 'srs_total'}).text ,
                'srs_offense': team.find(attrs = {'data-stat' : 'srs_offense'}).text,
                'srs_defense': team.find(attrs = {'data-stat' : 'srs_defense'}).text 
        }
        
        except:
            all_stats = {
                'team': '',
                'wins' :     '',
                'win_loss_perc': '',
                'points': '',
                'points_opp' : '',
                'points_diff' : '',
                'mov' : '',
                'sos_total' : '',
                'srs_total': '',
                'srs_offense': '',
                'srs_defense': '' 
            }
            # all_stats = None
            print('No data here')
        all_teams_data.append(all_stats)
    for team in nfc_teams:
        try:
            all_stats = {
                'team': team.find('th', attrs={"data-stat": "team"}).find('a').text,
                'wins' :     team.find(attrs = {'data-stat' : 'wins'}).text,
                'win_loss_perc': team.find(attrs = {'data-stat' : 'win_loss_perc'}).text,
                'points': team.find(attrs = {'data-stat' : 'points'}).text,
                'points_opp' : team.find(attrs = {'data-stat' : 'points_opp'}).text,
                'points_diff' : team.find(attrs = {'data-stat' : 'points_diff'}).text,
                'mov' : team.find(attrs = {'data-stat': 'mov'}).text,
                'sos_total' : team.find(attrs = {'data-stat' : 'sos_total'}).text,
                'srs_total': team.find(attrs = {'data-stat' : 'srs_total'}).text ,
                'srs_offense': team.find(attrs = {'data-stat' : 'srs_offense'}).text,
                'srs_defense': team.find(attrs = {'data-stat' : 'srs_defense'}).text 
        }
        
        except:
            all_stats = {
                'team': '',
                'wins' :     '',
                'win_loss_perc': '',
                'points': '',
                'points_opp' : '',
                'points_diff' : '',
                'mov' : '',
                'sos_total' : '',
                'srs_total': '',
                'srs_offense': '',
                'srs_defense': '' 
            }
            # all_stats = None
            print('No data here')
        all_teams_data.append(all_stats)
    return all_teams_data

In [60]:
get_all_teams(team_stats)

[<tr class="thead onecell"><td class="right left" colspan="12" data-stat="onecell"> AFC East</td></tr>, <tr><th class="left" csk="1" data-stat="team" scope="row"><a href="/teams/buf/2023.htm">Buffalo Bills</a>*</th><td class="right" data-stat="wins">11</td><td class="right" data-stat="losses">6</td><td class="right" data-stat="win_loss_perc">.647</td><td class="right" data-stat="points">451</td><td class="right" data-stat="points_opp">311</td><td class="right" data-stat="points_diff">140</td><td class="right" data-stat="mov">8.2</td><td class="right" data-stat="sos_total">-1.8</td><td class="right" data-stat="srs_total">6.5</td><td class="right" data-stat="srs_offense">4.1</td><td class="right" data-stat="srs_defense">2.3</td></tr>, <tr><th class="left" csk="2" data-stat="team" scope="row"><a href="/teams/mia/2023.htm">Miami Dolphins</a>+</th><td class="right" data-stat="wins">11</td><td class="right" data-stat="losses">6</td><td class="right" data-stat="win_loss_perc">.647</td><td cla

[{'team': '',
  'wins': '',
  'win_loss_perc': '',
  'points': '',
  'points_opp': '',
  'points_diff': '',
  'mov': '',
  'sos_total': '',
  'srs_total': '',
  'srs_offense': '',
  'srs_defense': ''},
 {'team': 'Buffalo Bills',
  'wins': '11',
  'win_loss_perc': '.647',
  'points': '451',
  'points_opp': '311',
  'points_diff': '140',
  'mov': '8.2',
  'sos_total': '-1.8',
  'srs_total': '6.5',
  'srs_offense': '4.1',
  'srs_defense': '2.3'},
 {'team': 'Miami Dolphins',
  'wins': '11',
  'win_loss_perc': '.647',
  'points': '496',
  'points_opp': '391',
  'points_diff': '105',
  'mov': '6.2',
  'sos_total': '-1.8',
  'srs_total': '4.4',
  'srs_offense': '7.6',
  'srs_defense': '-3.1'},
 {'team': 'New York Jets',
  'wins': '7',
  'win_loss_perc': '.412',
  'points': '268',
  'points_opp': '355',
  'points_diff': '-87',
  'mov': '-5.1',
  'sos_total': '-0.6',
  'srs_total': '-5.8',
  'srs_offense': '-6.7',
  'srs_defense': '0.9'},
 {'team': 'New England Patriots',
  'wins': '4',
  'win_

In [55]:
write_csv(get_all_teams(team_stats), '../data/2023_team_stats.csv')

[<tr class="thead onecell"><td class="right left" colspan="12" data-stat="onecell"> AFC East</td></tr>, <tr><th class="left" csk="1" data-stat="team" scope="row"><a href="/teams/buf/2023.htm">Buffalo Bills</a>*</th><td class="right" data-stat="wins">11</td><td class="right" data-stat="losses">6</td><td class="right" data-stat="win_loss_perc">.647</td><td class="right" data-stat="points">451</td><td class="right" data-stat="points_opp">311</td><td class="right" data-stat="points_diff">140</td><td class="right" data-stat="mov">8.2</td><td class="right" data-stat="sos_total">-1.8</td><td class="right" data-stat="srs_total">6.5</td><td class="right" data-stat="srs_offense">4.1</td><td class="right" data-stat="srs_defense">2.3</td></tr>, <tr><th class="left" csk="2" data-stat="team" scope="row"><a href="/teams/mia/2023.htm">Miami Dolphins</a>+</th><td class="right" data-stat="wins">11</td><td class="right" data-stat="losses">6</td><td class="right" data-stat="win_loss_perc">.647</td><td cla

In [45]:
import csv, time, sys, math
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from urllib.request import urlopen

html = urlopen(url)

In [57]:
#reusable URLs to help us get the different categories that we want. 

pro_football_all_seasons_url = 'https://www.pro-football-reference.com/years/NFL/'



In [58]:
qb_stats = get_stats(pro_football_all_seasons_url)
qb_stats.find('title')


<title>NFL Season By Season Team Offense | Pro-Football-Reference.com</title>

In [28]:
url = 'http://www.pro-football-reference.com/years/2023/#team_stats'

In [29]:
# pd.read_html(url)
pd.read_html(requests.get(url).text)


C:\Users\ckonj.MSI\AppData\Local\Temp\ipykernel_23724\1224355552.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(requests.get(url).text)


[                      Tm          W          L       W-L%         PF  \
 0               AFC East   AFC East   AFC East   AFC East   AFC East   
 1         Buffalo Bills*         11          6       .647        451   
 2        Miami Dolphins+         11          6       .647        496   
 3          New York Jets          7         10       .412        268   
 4   New England Patriots          4         13       .235        236   
 5              AFC North  AFC North  AFC North  AFC North  AFC North   
 6      Baltimore Ravens*         13          4       .765        483   
 7      Cleveland Browns+         11          6       .647        396   
 8   Pittsburgh Steelers+         10          7       .588        304   
 9     Cincinnati Bengals          9          8       .529        366   
 10             AFC South  AFC South  AFC South  AFC South  AFC South   
 11       Houston Texans*         10          7       .588        377   
 12  Jacksonville Jaguars          9          8    

In [39]:
response = requests.get(url)
soup = BeautifulSoup(response.text)
table = soup.find_all('table', {'id': 'team_stats'})

In [51]:
import requests
from bs4 import BeautifulSoup
from bs4 import Comment
import pandas as pd

response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

comments = soup.find_all(string=lambda text: isinstance(text, Comment))

tables = []
for each in comments:
    if 'table' in each:
        try:
            tables.append(pd.read_html(each, header=1)[0])
        except:
            continue

C:\Users\ckonj.MSI\AppData\Local\Temp\ipykernel_23724\550780763.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables.append(pd.read_html(each, header=1)[0])
C:\Users\ckonj.MSI\AppData\Local\Temp\ipykernel_23724\550780763.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables.append(pd.read_html(each, header=1)[0])
C:\Users\ckonj.MSI\AppData\Local\Temp\ipykernel_23724\550780763.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables.append(pd.read_html(each, header=1)[0])
C:\Users\ckonj.MSI\AppData\Local\Temp\ipykernel_23724\550780763.py:20: FutureWarning: Passing literal html to 'read_html' is deprecat

In [52]:
tables[3]

,Rk,Tm,G,PF,Yds,Ply,Y/P,TO,FL,1stD,...,Yds.2,TD.1,Y/A,1stD.2,Pen,Yds.3,1stPy,Sc%,TO%,EXP
0,1.0,Dallas Cowboys,17.0,509.0,6317.0,1122.0,5.6,16.0,6.0,385.0,...,1920.0,14.0,4.1,113.0,115.0,964.0,43.0,50.3,8.9,192.72
1,2.0,Miami Dolphins,17.0,496.0,6822.0,1053.0,6.5,25.0,10.0,360.0,...,2308.0,27.0,5.1,113.0,97.0,767.0,24.0,43.5,13.4,128.90
2,3.0,San Francisco 49ers,17.0,491.0,6773.0,1024.0,6.6,18.0,6.0,383.0,...,2389.0,27.0,4.8,147.0,101.0,933.0,29.0,45.3,10.1,268.60
3,4.0,Baltimore Ravens,17.0,483.0,6296.0,1076.0,5.9,19.0,12.0,360.0,...,2661.0,26.0,4.9,145.0,102.0,955.0,35.0,43.1,9.6,138.81
4,5.0,Detroit Lions,17.0,461.0,6712.0,1137.0,5.9,23.0,11.0,375.0,...,2311.0,27.0,4.6,124.0,97.0,843.0,23.0,40.6,11.8,191.13
5,6.0,Buffalo Bills,17.0,451.0,6366.0,1115.0,5.7,28.0,10.0,381.0,...,2212.0,22.0,4.3,158.0,106.0,883.0,24.0,41.4,14.9,197.57
6,7.0,Philadelphia Eagles,17.0,433.0,6024.0,1112.0,5.4,28.0,12.0,377.0,...,2190.0,22.0,4.3,149.0,95.0,785.0,31.0,42.9,15.3,100.97
7,8.0,Los Angeles Rams,17.0,404.0,6108.0,1094.0,5.6,18.0,5.0,351.0,...,2045.0,18.0,4.3,110.0,89.0,720.0,35.0,41.1,9.2,133.32
8,9.0,New Orleans Saints,17.0,402.0,5732.0,1121.0,5.1,18.0,7.0,337.0,...,1742.0,13.0,3.6,112.0,96.0,846.0,26.0,36.6,8.2,46.88
9,10.0,Indianapolis Colts,17.0,396.0,5725.0,1094.0,5.2,22.0,12.0,324.0,...,2059.0,19.0,4.3,113.0,95.0,685.0,33.0,35.7,9.2,30.12


In [32]:
afc_standings = pd.read_html(url, attrs={'id': 'AFC'})[0]
nfc_standings = pd.read_html(url, attrs={'id': 'NFC'})[0]
passing = pd.read_html(url, header= 1, attrs={'id': 'returns'})[0]

ValueError: No tables found

In [10]:
afc_standings

,Tm,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS
0,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East
1,New England Patriots*,12,4,0,.750,468,313,155,9.7,1.3,10.9,7.5,3.5
2,Buffalo Bills,9,7,0,.563,343,289,54,3.4,1.6,4.9,-0.4,5.3
3,Miami Dolphins,8,8,0,.500,388,373,15,0.9,1.6,2.6,2.9,-0.4
4,New York Jets,4,12,0,.250,283,401,-118,-7.4,2.3,-5.0,-4.0,-1.0
5,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North
6,Pittsburgh Steelers*,11,5,0,.688,436,368,68,4.3,-2.0,2.2,4.4,-2.1
7,Cincinnati Bengals+,10,5,1,.656,365,344,21,1.3,-0.6,0.7,-0.5,1.3
8,Baltimore Ravens+,10,6,0,.625,409,302,107,6.7,-2.1,4.6,1.8,2.8
9,Cleveland Browns,7,9,0,.438,299,337,-38,-2.4,-1.5,-3.9,-4.8,0.9


In [13]:
nfc_standings

,Tm,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS
0,NFC East,NFC East,NFC East,NFC East,NFC East,NFC East,NFC East,NFC East,NFC East,NFC East,NFC East,NFC East,NFC East
1,Dallas Cowboys*,12,4,0,.750,467,352,115,7.2,-1.8,5.4,5.3,0.1
2,Philadelphia Eagles,10,6,0,.625,474,400,74,4.6,-0.7,3.9,6.6,-2.7
3,New York Giants,6,10,0,.375,380,400,-20,-1.3,-0.4,-1.7,0.8,-2.5
4,Washington Redskins,4,12,0,.250,301,438,-137,-8.6,-0.2,-8.7,-4.0,-4.7
5,NFC North,NFC North,NFC North,NFC North,NFC North,NFC North,NFC North,NFC North,NFC North,NFC North,NFC North,NFC North,NFC North
6,Green Bay Packers*,12,4,0,.750,486,348,138,8.6,-0.3,8.3,7.9,0.4
7,Detroit Lions+,11,5,0,.688,321,282,39,2.4,-0.4,2.1,-3.2,5.2
8,Minnesota Vikings,7,9,0,.438,325,343,-18,-1.1,-0.5,-1.7,-2.8,1.1
9,Chicago Bears,5,11,0,.313,319,442,-123,-7.7,1.0,-6.7,-2.0,-4.7


In [16]:
passing

NameError: name 'passing' is not defined